In [8]:
def get_alphabet_choice(text):
  if not text:
    return "N/A"
  # pdb.set_trace()
  match = re.findall(r"answer is \((A|B|C|D|E|F)\)", text)
  if match:
    return match[-1]
  else:
    match = re.findall(r"\((A|B|C|D|E|F)\)", text)
    return match[-1] if match else "N/A"

text = 'Read the problem carefully. Break it into small sub‑problems. Solve each sub‑problem step by step. Double‑check the answers.\n\nGiven the following question and 4 candidate answers (A, B, C, D), choose the best answer.\nQuestion: An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?\nA. Planetary density will decrease.\nB. Planetary years will become longer.\nC. Planetary days will become shorter.\nD. Planetary gravity will become stronger.\nPlease conclude with your choice. Your response should end with "The best answer is [LETTER]" where LETTER is one of A, B, C, D.\n\nThe best answer is C. Planetary days will become shorter.\n\nExplanation:\n\n1. The question asks about the effect of an increase in rotation of a planet.\n2. Rotation refers to the spinning motion of the planet on its axis.\n3. A day on a planet is defined as the time it takes for the planet to complete one full rotation on its axis.\n4. Therefore, an increase in rotation would result in a decrease in the time it takes for the planet to complete one full rotation, which means the planet\'s days will become shorter.\n5. The other options do not directly relate to the increase in rotation:\n   a. Planetary density is not directly related to rotation.\n   b. Planetary years are not directly related to rotation.\n   c. Planetary gravity is not directly related to rotation, but it could be affected by changes in the planet\'s mass or shape, which are not mentioned in the question. However, an increase in rotation would not necessarily cause an increase in gravity.\n\nTherefore, the best answer is C. Planetary days will become shorter.'
get_alphabet_choice(text)

'N/A'

In [15]:
import re

latex_string = r"\boxed{\left(\frac{60}{65}, \frac{20}{13}\right)}"
match = re.compile("\\boxed\{(.*?)\}").findall(latex_string)[-1]
# match = re.search(pattern, latex_string)

if match:
    extracted_string = match.group(1)
    print(extracted_string)
else:
    print("No match found.")

IndexError: list index out of range

In [2]:
#!/usr/bin/env python3
# test_two_lora_cuda_v2.py  – multi-epoch, harder prompts
# --------------------------------------------------------
import torch, random, math
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig

BASE_MODEL   = "hf-internal-testing/tiny-random-GPT2LMHeadModel"
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
TEMPERATURE  = 1.0
LAMBDA_KL    = 0.5
EPOCHS       = 5
BATCH_SIZE   = 4               # keep small – tiny model has 50257-token vocab

# -- build model + two LoRA adapters ------------------------
cfg = dict(
    r=8, lora_alpha=32, lora_dropout=0.05, target_modules=["c_attn"]
)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL).to(DEVICE)
model.add_adapter(LoraConfig(**cfg), adapter_name="A")
model.add_adapter(LoraConfig(**cfg), adapter_name="B")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token  # needed for padding

# -- pool of “difficult” prompts ----------------------------
PROMPT_POOL = [
    # odd capitalization, rare words, and LaTeX
    "Incongruous rhythms of γ-oscillation juxtapose with d_{i,j}=∑_k α_k",
    # mixed languages & emoji
    "Quand je dis « superposition », 我并不是 🤔 talking about furniture.",
    # long numeric / scientific string
    "The half-life of ¹³⁷Cs is 30.17 years; estimate its activity after 9.2×10⁸ seconds.",
    # code-like fragment
    "def paradox(x): return (lambda y: y(y))(lambda y: y(y))(x)",
    # contradiction
    "The smallest prime number larger than 100 is obviously 99.",
    # heavy punctuation
    "Wait—did he just say, “algorithmic *entropy* equals ∑-p log₂ p”?!",
    # archaic phrasing
    "Hark! what resplendent algorithm bewitches yon silicon steed?",
    # deliberately misspelled tokens
    "Qantum mechanicz deskribe probabilistic superpostions.",
]

def random_batch(batch_size):
    prompts = random.sample(PROMPT_POOL, batch_size)
    data = tokenizer(prompts, return_tensors="pt", padding=True).to(DEVICE)
    data["labels"] = data["input_ids"].clone()
    return data

def forward_two_adapters(m, **kw):
    m.set_adapter("A"); out_a = m(**kw)
    m.set_adapter("B"); out_b = m(**kw)
    return out_a.logits, out_b.logits

def grad_mag(tag):
    g = [p.grad.abs().mean() for n,p in model.named_parameters()
         if tag in n and p.grad is not None]
    return float(torch.stack(g).mean()) if g else 0.0

# --------------- training loop -----------------------------
model.train()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(1, EPOCHS+1):
    batch = random_batch(BATCH_SIZE)
    logits_a, logits_b = forward_two_adapters(model, **batch)

    loss_ce_a = F.cross_entropy(
        logits_a.view(-1, logits_a.size(-1)),
        batch["labels"].view(-1)
    )
    loss_ce_b = F.cross_entropy(
        logits_b.view(-1, logits_b.size(-1)),
        batch["labels"].view(-1)
    )

    loss_kl = F.kl_div(
        F.log_softmax(logits_a / TEMPERATURE, dim=-1),
        F.softmax(logits_b.detach() / TEMPERATURE, dim=-1),
        reduction="batchmean"
    )
    loss = loss_ce_a + loss_ce_b + LAMBDA_KL * loss_kl 

    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

    print(f"[epoch {epoch}] "
          f"CE_A={loss_ce_a.item():.3e}  CE_B={loss_ce_b.item():.3e} "
          f"|∇|_A={grad_mag('lora_A'):.3e}  |∇|_B={grad_mag('lora_B'):.3e}")

/project/uvadm/zhenyu/miniconda3/envs/distill/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/project/uvadm/zhenyu/miniconda3/envs/distill/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/project/uvadm/zhenyu/miniconda3/envs/distill/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


[epoch 1] CE_A=6.520e+00  CE_B=6.527e+00 |∇|_A=0.000e+00  |∇|_B=4.512e-04
[epoch 2] CE_A=6.528e+00  CE_B=6.530e+00 |∇|_A=3.428e-06  |∇|_B=4.342e-04
[epoch 3] CE_A=6.528e+00  CE_B=6.528e+00 |∇|_A=7.188e-06  |∇|_B=5.402e-04
[epoch 4] CE_A=6.535e+00  CE_B=6.535e+00 |∇|_A=1.286e-05  |∇|_B=6.354e-04
[epoch 5] CE_A=6.531e+00  CE_B=6.530e+00 |∇|_A=1.190e-05  |∇|_B=4.699e-04


NameError: name 'Dataset' is not defined

In [3]:
####### Tokenizer & Base Model #######
print("Loading tokenizer and base model...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)
for p in base_model.parameters():
    p.requires_grad = False
base_model.eval()

Loading tokenizer and base model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.09s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [ ]:
######## Collate Function ########
def collate_fn(batch):
    ins, cots = zip(*batch)
    out = {k: [] for k in [
        'enc_ids', 'enc_mask', 'gen_ids', 'gen_mask', 'labels'
    ]}

    for inp, cot in zip(ins, cots):
        e = [tokenizer(inp + c, return_tensors='pt',
                        truncation=True, max_length=1024) for c in cot]
        g = tokenizer(inp, return_tensors='pt',
                        padding='max_length',
                        truncation=True, max_length=1024)
        L = len(g['input_ids'][0])

        lab = [tokenizer(inp + c, return_tensors='pt',
                        truncation=True, max_length=1024)['input_ids'] for c in cot]
                        
        for i in range(len(lab)):
            lab[i][:, L:] = -100

        for k, v in zip(
            ['enc_ids1', 'enc_mask1', 'enc_ids2', 'enc_mask2', 'gen_ids', 'gen_mask', 'labels1', 'labels2'],
            [e1['input_ids'], e1['attention_mask'], e2['input_ids'], e2['attention_mask'], g['input_ids'], g['attention_mask'], lab1, lab2]
        ):
            out[k].append(v)

    for k in out: out[k] = torch.cat(out[k], dim=0)
    return out


In [ ]:
####### add datasets ######
train_set = ReasoningDataset(...)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=4)
test_set = EvalDataset(load_jsonl(TEST_PATH))
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=4)

######### Model #########
student_1 = get_peft_model(
    copy.deepcopy(base_model),
    LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=['q_proj', 'v_proj']
    )
)
student_2 = get_peft_model(
    copy.deepcopy(base_model),
    LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=['q_proj', 'v_proj']
    )
)

# router
class Router():
    """
    input: hiddens n * d
    output: logits num_stu * 1
    """
    def __init__(self):
        pass
    
    def forward(self, hiddens):
        return logits

# encoder
class CoTEncoder():
    """
    input: prompt + cot
    output: hiddens n * d
    """
    def __init__(self, model):
        self.model = model
    
    def forward(self, input_ids, attention_mask):
        return hiddens

class WeightLearner():
    """
    input: hiddens n * d
    output: logits 1
    """
    def __init__(self, hiddens):
        self.hiddens = hiddens
    
    def forward(self, input_ids, attention_mask):
        return logits

class DistillModel():
    """input: prompt + cot
    output: loss
    """
    def __init__(self, model):
        self.model = model
        self.cot_encoder = CoTEncoder(model)
        self.router = Router(model)
        self.weight_learner = WeightLearner(model)
        self.temperature = TEMPERATURE
        self.kl_weight = KL_WEIGHT
        self.num_stu = 2
    def forward(self, inputs):
        """
        inputs: dict with keys
            enc_ids1, enc_mask1, enc_ids2, enc_mask2, gen_ids, gen_mask, labels1, labels2
        """
        # forward pass
        hiddens_1 = self.cot_encoder.forward(inputs['enc_ids1'], inputs['enc_mask1'])
        hiddens_2 = self.cot_encoder.forward(inputs['enc_ids2'], inputs['enc_mask2'])
        router_logits_1 = self.router.forward(hiddens_1)
        router_logits_2 = self.router.forward(hiddens_2)

        # choose which student to use given router_logits
        logits_1 = 

        logits = self.router.forward(inputs['enc_ids1'], inputs['enc_mask1'])
        loss = F.cross_entropy(logits, inputs['labels1'])
        # KL loss
        hiddens = self.cot_encoder.forward(inputs['enc_ids2'], inputs['enc_mask2'])
        logits = self.weight_learner.forward(hiddens)
        loss += self.kl_weight * F.kl_div(
            F.log_softmax(logits / self.temperature, dim=-1),
            F.softmax(logits / self.temperature, dim=-1),
            reduction='batchmean'
        )
        return loss


class Trainer():
    def __init__(self, model, train_loader, test_loader):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
    
    def train(self):
        self.model.train()
        for epoch in range(NUM_EPOCHS):
            for batch in tqdm(self.train_loader):
                # forward pass
                logits = self.model.forward(batch['enc_ids1'], batch['enc_mask1'])
                loss = F.cross_entropy(logits, batch['labels1'])
                loss.backward()
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
    
    def evaluate(self):
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(self.test_loader):
                # forward pass
                logits = self.model.forward(batch['enc_ids1'], batch['enc_mask1'])
                loss = F.cross_entropy(logits, batch['labels1'])
    



In [1]:
# main.py  ── two-student MoE with CoT routing + weighted ensemble KL
import os, json, random, re, copy, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# ──────────────────────── CONFIG ────────────────────────
DATA_PATH       = "data/gsm8k/train/train_cot_distill.jsonl"
DEV_PATH        = "data/gsm8k/train/test_cot_distill.jsonl"
TEST_PATH       = "data/cot_response.jsonl"
BASE_MODEL      = "mistralai/Mistral-7B-Instruct-v0.1"
BATCH_SIZE      = 4
NUM_EPOCHS      = 2
LR              = 1e-5

alpha           = 0.5     # unused (was ensemble weight in baseline)
temperature     = 1.0
kl_weight       = 0.5
USE_KL_DISTILLATION = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True      # (optional) speed

# ──────────────────────── HELPERS ────────────────────────
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                json_object = json.loads(line)
                data.append(json_object)
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line.strip()}")
    return data

# ──────────────────────── DATASETS ────────────────────────
class ReasoningDataset(Dataset):
    """
    Returns: instruction, two sampled CoTs, label
    The two CoTs are later routed to two students.
    """
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        instruction = item['prompt']
        cot = item['messages'][1]['content']
        cot1, cot2 = [cot, cot]
        return instruction, cot1, cot2, None

class EvalDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        d = self.data[idx]
        return d["question"], d["answer"]

# ──────────────────────── TOKENIZER & BASE MODEL ────────────────────────
print("Loading tokenizer & frozen backbone …")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)
for p in base_model.parameters(): p.requires_grad = False
base_model.eval()

# ──────────────────────── COLLATE FN ────────────────────────
def collate_fn(batch):
    ins, cot1, cot2, _ = zip(*batch)                 # we don't need labels here
    out = {k: [] for k in [
        "enc_ids1","enc_mask1","enc_ids2","enc_mask2",
        "gen_ids","gen_mask","labels1","labels2"
    ]}

    for inp, c1, c2 in zip(ins, cot1, cot2):
        # — tokens for the encoder (instruction + CoT) —
        e1 = tokenizer(inp + c1, return_tensors="pt",
                       truncation=True, max_length=1024)
        e2 = tokenizer(inp + c2, return_tensors="pt",
                       truncation=True, max_length=1024)

        # — instruction-only prompt for generation —
        g  = tokenizer(inp, return_tensors="pt",
                       padding="max_length",
                       truncation=True, max_length=1024)
        L  = len(g["input_ids"][0])        # prompt length

        # — labels (mask out prompt tokens) —
        lab1 = tokenizer(inp + c1, return_tensors="pt",
                         padding="max_length",
                         truncation=True, max_length=1024)["input_ids"]
        lab2 = tokenizer(inp + c2, return_tensors="pt",
                         padding="max_length",
                         truncation=True, max_length=1024)["input_ids"]
        lab1[:, :L] = -100
        lab2[:, :L] = -100

        for k, v in zip(
            ["enc_ids1","enc_mask1","enc_ids2","enc_mask2",
             "gen_ids","gen_mask","labels1","labels2"],
            [e1["input_ids"], e1["attention_mask"],
             e2["input_ids"], e2["attention_mask"],
             g["input_ids"],  g["attention_mask"],
             lab1, lab2]):
            out[k].append(v)

    for k in out: out[k] = torch.cat(out[k], dim=0)
    return out

# ──────────────────────── NEW MODULES ────────────────────────
class CotEncoder(nn.Module):
    """Frozen encoder that pools last hidden state."""
    def __init__(self, backbone, pool="mean"):
        super().__init__()
        self.backbone, self.pool = backbone, pool
    @torch.no_grad()
    def forward(self, input_ids, attention_mask):
        h = self.backbone.model(
            input_ids     = input_ids,
            attention_mask= attention_mask,
            output_hidden_states=True,
            return_dict=True
        ).hidden_states[-1]                     # [B,T,d]
        if self.pool == "mean":
            num = (h * attention_mask.unsqueeze(-1)).sum(1)
            den = attention_mask.sum(1, keepdim=True)
            return (num / den).float()          # [B,d]
        return h[:,0].float()

class Router(nn.Module):
    """Two-way hard router with straight-through Gumbel-Softmax."""
    def __init__(self, d_model, tau=1.0):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.GELU(),
            nn.Linear(d_model//2, 2)
        )
        self.tau = tau
    def forward(self, h):       # h:[B,d]
        return F.gumbel_softmax(self.mlp(h), tau=self.tau, hard=True)

class WeightLearner(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.fc = nn.Linear(d_model, 1)
    def forward(self, h):       # h:[B,d]
        return torch.sigmoid(self.fc(h)).squeeze(-1)   # [B]

# ──────────────────────── LoRA STUDENTS ────────────────────────
def make_student(backbone):
    model = copy.deepcopy(backbone)
    cfg   = LoraConfig(
        r=8, lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none", task_type=TaskType.CAUSAL_LM
    )
    model = get_peft_model(model, cfg)
    model.print_trainable_parameters()
    return model

print("Building two LoRA students …")
student1, student2 = make_student(base_model), make_student(base_model)

# ──────────────────────── INSTANTIATE NEW UTILS ────────────────────────
encoder     = CotEncoder(base_model).to(DEVICE)
router      = Router(base_model.config.hidden_size).to(DEVICE)
weight_net  = WeightLearner(base_model.config.hidden_size).to(DEVICE)

# ──────────────────────── DATA LOADERS ────────────────────────
train_set = ReasoningDataset(load_data(DATA_PATH), tokenizer)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_fn)

# ──────────────────────── OPTIMIZER ────────────────────────
trainables = (list(student1.parameters()) +
              list(student2.parameters()) +
              list(router.parameters())   +
              list(weight_net.parameters()))
optimizer = torch.optim.AdamW(trainables, lr=LR)

# ──────────────────────── TRAIN ────────────────────────
for epoch in range(1, NUM_EPOCHS+1):
    prog = tqdm(train_loader, desc=f"Epoch {epoch}", dynamic_ncols=True)
    for batch in prog:
        # ─ move to device ─
        for k in batch: batch[k] = batch[k].to(DEVICE)
        gen_ids, labels1, labels2 = batch["gen_ids"], batch["labels1"], batch["labels2"]

        # ─ encode both CoTs ─
        h1 = encoder(batch["enc_ids1"], batch["enc_mask1"])
        h2 = encoder(batch["enc_ids2"], batch["enc_mask2"])

        # ─ routing decisions ─
        gate1 = router(h1)          # [B,2] one-hot
        gate2 = router(h2)
        m1 = (gate1[:,0] | gate2[:,0]).bool()   # mask for student-1
        m2 = (gate1[:,1] | gate2[:,1]).bool()   # mask for student-2

        optimizer.zero_grad()

        # ─ student-1 forward ─
        if m1.any():
            out1 = student1(input_ids=gen_ids[m1], labels=labels1[m1])
            logits1 = torch.zeros((*labels1.shape, student1.config.vocab_size),
                                  dtype=out1.logits.dtype, device=DEVICE)
            logits1[m1] = out1.logits
            ce1 = out1.loss
        else:
            logits1 = torch.zeros((*labels1.shape, base_model.config.vocab_size),
                                  dtype=torch.float16, device=DEVICE)
            ce1 = torch.tensor(0., device=DEVICE)

        # ─ student-2 forward ─
        if m2.any():
            out2 = student2(input_ids=gen_ids[m2], labels=labels2[m2])
            logits2 = torch.zeros_like(logits1)
            logits2[m2] = out2.logits
            ce2 = out2.loss
        else:
            logits2 = torch.zeros_like(logits1)
            ce2 = torch.tensor(0., device=DEVICE)

        # ─ learn sample weight w ∈ (0,1) ─
        w = weight_net((h1 + h2) / 2).view(-1,1,1)     # [B,1,1]
        logits_ens = w * logits1 + (1 - w) * logits2

        # ─ KL divergence against ensemble ─
        if USE_KL_DISTILLATION:
            logp1 = F.log_softmax(logits1 / temperature, dim=-1)
            logp2 = F.log_softmax(logits2 / temperature, dim=-1)
            logpE = F.log_softmax(logits_ens.detach() / temperature, dim=-1)

            tok_mask = (labels1 != -100)
            kl1 = F.kl_div(logp1, logpE, reduction="none",
                           log_target=True).sum(-1)[tok_mask].mean()
            kl2 = F.kl_div(logp2, logpE, reduction="none",
                           log_target=True).sum(-1)[tok_mask].mean()
            loss = ce1 + ce2 + kl_weight * (kl1 + kl2)
        else:
            kl1 = kl2 = torch.tensor(0., device=DEVICE)
            loss = ce1 + ce2

        loss.backward()
        optimizer.step()

        prog.set_postfix({
            "loss": f"{loss.item():.4f}",
            "CE1":  f"{ce1.item():.3f}",
            "CE2":  f"{ce2.item():.3f}",
            "KL":   f"{(kl1+kl2).item():.3f}"
        })

# ──────────────────────── SIMPLE EVAL (router → one student) ────────────────────────
def extract_number(text):
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", text)
    return nums[-1] if nums else None

@torch.no_grad()
def evaluate(dataset_path):
    data = load_data(dataset_path)
    loader = DataLoader(EvalDataset(data), batch_size=1)
    correct = total = 0
    for ins, label in tqdm(loader, desc=f"Eval {os.path.basename(dataset_path)}"):
        prompt = ins[0]
        e = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        h = encoder(e["input_ids"], e["attention_mask"])
        gate = router(h)[0]              # [2]
        if gate[0] == 1:
            outs = student1.generate(**e, max_length=256)
        else:
            outs = student2.generate(**e, max_length=256)
        pred_ans = extract_number(tokenizer.decode(outs[0], skip_special_tokens=True))
        gold_ans = extract_number(label[0])
        correct += (pred_ans == gold_ans)
        total   += 1
    acc = correct / total
    print(f"Accuracy: {acc:.4f}")

evaluate(TEST_PATH)

/project/uvadm/zhenyu/miniconda3/envs/distill/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer & frozen backbone …


Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.11s/it]


Building two LoRA students …
trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470
trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


Epoch 1:   0%|          | 0/1772 [00:00<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 159 but got size 287 for tensor number 1 in the list.